# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:

1. Name: Reddy Balaji .C (BITS ID: 2023AC05862); Email: 2023ac05862@wilp.bits-pilani.ac.in

    Contribution- 100%

4. Name: Saurabh Jalendra (BITS ID: 2023AC05912); Email: 2023ac05912@wilp.bits-pilani.ac.in

   Contribution- 100%

6. Name: Tushar Shandilya (BITS ID: 2023AC05573); Email: 2023ac05573@wilp.bits-pilani.ac.in

   Contribution- 100%

8. Name:  Bandana Kumari (BITS ID: 2023AC05879); Email: 2023ac05879@wilp.bits-pilani.ac.in

   Contribution- 100%

10. Name: Monica Malik (BITS ID: 2023AC05875); Email: 2023ac05875@wilp.bits-pilani.ac.in

    Contribution- 100%

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

# List the PEAS decription of the problem here in this markdown block:

High level PEAS is shared here and the detailing of each of the line item is in the supporting word document shared.

1.	Performance Measure:

    The performance of the GPS Navigation Agent is evaluated based on the following criteria:

        - Safely Reaching the Destination
        - Shortest Path 
        - Quickest Route 
        - Safest Path
        - Minimized Cost

2.	Environment:

The environment in which the GPS Navigation Agent operates is a structured grid-based representation of a city including the following elements:

        - Grid-Based City Map
        - Buildings
        - Blocked Paths
        - Roadblocks
        - Traffic Conditions

3.	Actuator:

The actuator defines the actions that the GPS Navigation Agent can perform within the environment:

        - Movement Directions
        - Path Execution

4.	Sensor:

The sensor system provides the GPS Navigation Agent with real-time information about its environment allowing it to make informed decisions:

        - City Map Information
        - Environmental Sensors
        - Proximity Sensors
        - Real-Time Grid Status


Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)
def set_initial_state():
    start = (0, 0)  # Coordinates of 'S' (starting point)
    goal = (5, 5)   # Coordinates of 'G' (goal point)
    return start, goal

start, goal = set_initial_state()
print("Initial State:")
print(f"Start: {start}, Goal: {goal}")


Initial State:
Start: (0, 0), Goal: (5, 5)


In [2]:
# Code Block : Set the matrix for transition & cost (as relevant for the given problem)
# Define the grid representing the city layout
grid = [
    ['S', '.', 'X', 'X', '.', '.'],
    ['B', '.', 'X', '.', '.', '.'],
    ['.', '.', '.', '.', 'X', '.'],
    ['B', '.', 'X', '.', '.', 'B'],
    ['.', '.', '.', '.', 'B', '.'],
    ['.', 'B', '.', 'X', '.', 'G']
]
# Define costs for adjacency to buildings and roadblocks
adjacent_building_cost = -5   # Points deducted for adjacent building
adjacent_roadblock_cost = +3  # Points added for adjacent road block
diagonal_movement_penalty = 3  # Cost for diagonal movement

print("Grid Definition:")
for row in grid:
    print(row)


Grid Definition:
['S', '.', 'X', 'X', '.', '.']
['B', '.', 'X', '.', '.', '.']
['.', '.', '.', '.', 'X', '.']
['B', '.', 'X', '.', '.', 'B']
['.', '.', '.', '.', 'B', '.']
['.', 'B', '.', 'X', '.', 'G']


In [3]:
# Code Block : Write function to design the Transition Model/Successor function.
# Ideally this would be called while search algorithms are implemented
# Function to calculate additional cost for adjacency
def calculate_adjacent_cost(x, y):
    adjacent_cost = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < len(grid) and 0 <= new_y < len(grid[0]):
            if grid[new_x][new_y] == 'B':
                adjacent_cost += adjacent_building_cost
            elif grid[new_x][new_y] == 'X':
                adjacent_cost += adjacent_roadblock_cost
    return adjacent_cost

# Function to get valid neighbors (successor function)
def get_neighbors(position, visited):
    x, y = position
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    # Check for valid straight moves
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if (0 <= new_x < len(grid) and 
            0 <= new_y < len(grid[0]) and 
            grid[new_x][new_y] != 'X' and 
            grid[new_x][new_y] != 'B' and 
            (new_x, new_y) not in visited):
            neighbors.append((new_x, new_y, 1 + calculate_adjacent_cost(new_x, new_y)))

    # Check for diagonal moves if no valid straight moves are found
    if not neighbors:
        diagonal_directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]  # Diagonals
        for dx, dy in diagonal_directions:
            new_x, new_y = x + dx, y + dy
            if (0 <= new_x < len(grid) and 
                0 <= new_y < len(grid[0]) and 
                grid[new_x][new_y] != 'X' and 
                grid[new_x][new_y] != 'B' and 
                (new_x, new_y) not in visited):
                neighbors.append((new_x, new_y, diagonal_movement_penalty + calculate_adjacent_cost(new_x, new_y)))

    return neighbors


In [4]:
# Code block : Write function to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
# Function to check if the current state is the goal state
def is_goal(state, goal):
    return state == goal


### 2.	Definition of Algorithm 1 (Recursive Best First Search - RBFS)

In [5]:
#Code Block : Function for algorithm 1 implementation
import time

def heuristic(node, goal):
    # Heuristic function calculating Manhattan distance to the goal
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])
def recursive_best_first_search(start, goal):
    expanded_nodes = 0  # Counter for expanded nodes
    visited = set()      # Set to track visited nodes

    # A dictionary to keep track of costs to reach each node
    cost_at_node = {start: 0}

    def rbf_search(node, g, h):
        nonlocal expanded_nodes
        expanded_nodes += 1  # Increment expanded node count
        visited.add(node)     # Mark the current node as visited
        f = g + h  # Calculate total cost
        if is_goal(node, goal):
            return g, [node]  # If goal reached, return cost and path
        
        best = float('inf')  # Initialize best cost as infinity
        best_path = []
        for neighbor in get_neighbors(node, visited):
            neighbor_pos, cost = neighbor[:2], neighbor[2]
            g_next = g + cost  # Update cost to reach this neighbor
            cost_at_node[neighbor_pos] = g_next  # Store cost to reach this neighbor
            h_next = heuristic(neighbor_pos, goal)  # Heuristic for neighbor
            result, path = rbf_search(neighbor_pos, g_next, h_next)  # Recursive call
            if result < best:
                best = result  # Update best cost
                best_path = path  # Update best path

        return best, [node] + best_path  # Return best cost and path

    start_time = time.time()  # Start timing the search
    total_cost, path = rbf_search(start, 0, heuristic(start, goal))  # Start recursive search
    end_time = time.time()  # End timing

    print("Algorithm 1 (Recursive Best First Search):")
    print(f"Total Cost: {total_cost}, Expanded Nodes: {expanded_nodes}, Time: {end_time - start_time:.10} seconds")
    
    print("Path with Costs and Manhattan Distances:")
    for node in path:
        manhattan_distance = heuristic(node, goal)
        cost = cost_at_node[node]  # Get the cost to reach this node
        print(f"Node: {node}, Cost: {cost}, Manhattan Distance: {manhattan_distance}")

    return total_cost, path, expanded_nodes, end_time - start_time


### 3.	Definition of Algorithm 2 (Uniform Cost Search-UCS)

In [6]:
# Code Block : Function for algorithm 2 implementation
import heapq

def uniform_cost_search(start, goal):
    expanded_nodes = 0   # Counter for expanded nodes
    visited = set()      # Set to track visited nodes
    priority_queue = [(0, start)]     # Initialize priority queue with the starting node
    came_from = {start: None}   # To reconstruct the path
    cost_so_far = {start: 0}    # To track cost to reach each node

    start_time = time.time()    # Start timing the search

    while priority_queue:
        current_cost, current_node = heapq.heappop(priority_queue)   # Get the node with lowest cost
        expanded_nodes += 1   # Increment expanded node count
        visited.add(current_node)   # Mark the current node as visited

        if is_goal(current_node, goal):
            break   # Stop if goal is reached

        for neighbor in get_neighbors(current_node, visited):
            neighbor_pos, cost = neighbor[:2], neighbor[2]
            new_cost = current_cost + cost   # Calculate new cost
            if neighbor_pos not in cost_so_far or new_cost < cost_so_far[neighbor_pos]:
                cost_so_far[neighbor_pos] = new_cost
                heapq.heappush(priority_queue, (new_cost, neighbor_pos))   # Add to priority queue
                came_from[neighbor_pos] = current_node    # Track where we came from

    end_time = time.time()   # End timing

    # Reconstruct path
    path = []
    path_costs = []
    current = goal   # Start from the goal
    while current is not None:
        path.append(current)   # Build the path backward
        path_costs.append(cost_so_far[current])
        current = came_from[current]
    path.reverse()   # Reverse the path to correct order
    path_costs.reverse()

    print("Algorithm 2 (Uniform Cost Search):")
    print(f"Total Cost: {cost_so_far[goal]}, Path: {list(zip(path, path_costs))}, Expanded Nodes: {expanded_nodes}, Time: {end_time - start_time:.10f} seconds")
    return cost_so_far[goal], path, path_costs, expanded_nodes, end_time - start_time


### DYNAMIC INPUT


IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [7]:
#Code Block : Function & call to get inputs (start/end state)
def get_input():
    return start, goal

start, goal = get_input()
print(f"Start: {start}, Goal: {goal}")


Start: (0, 0), Goal: (5, 5)


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [8]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
cost_1, path_1, expanded_nodes_1, time_1 = recursive_best_first_search(start, goal)
print("\nFinal Results for Algorithm 1:")
print(f"Total Cost: {cost_1}, Path: {path_1}, Expanded Nodes: {expanded_nodes_1}, Time: {time_1:.10f} seconds")


Algorithm 1 (Recursive Best First Search):
Total Cost: 17, Expanded Nodes: 29, Time: 0.0 seconds
Path with Costs and Manhattan Distances:
Node: (0, 0), Cost: 0, Manhattan Distance: 10
Node: (0, 1), Cost: 4, Manhattan Distance: 9
Node: (1, 1), Cost: 3, Manhattan Distance: 8
Node: (2, 1), Cost: 4, Manhattan Distance: 7
Node: (3, 1), Cost: 3, Manhattan Distance: 6
Node: (4, 1), Cost: -1, Manhattan Distance: 5
Node: (4, 2), Cost: 3, Manhattan Distance: 4
Node: (5, 2), Cost: 2, Manhattan Distance: 3
Node: (4, 3), Cost: 2, Manhattan Distance: 3
Node: (3, 3), Cost: 7, Manhattan Distance: 4
Node: (2, 3), Cost: 11, Manhattan Distance: 5
Node: (1, 3), Cost: 18, Manhattan Distance: 6
Node: (1, 4), Cost: 22, Manhattan Distance: 5
Node: (0, 4), Cost: 26, Manhattan Distance: 6
Node: (0, 5), Cost: 27, Manhattan Distance: 5
Node: (1, 5), Cost: 23, Manhattan Distance: 4
Node: (2, 5), Cost: 27, Manhattan Distance: 3
Node: (3, 4), Cost: 1, Manhattan Distance: 3
Node: (4, 5), Cost: 16, Manhattan Distance:

In [9]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
cost_2, path_2, path_costs_2, expanded_nodes_2, time_2 = uniform_cost_search(start, goal)
print("\nFinal Results for Algorithm 2:")
print(f"Total Cost: {cost_2}, Path: {list(zip(path_2, path_costs_2))}, Expanded Nodes: {expanded_nodes_2}, Time: {time_2:.10f}")


Algorithm 2 (Uniform Cost Search):
Total Cost: -13, Path: [((0, 0), 0), ((0, 1), 4), ((1, 1), 3), ((2, 1), 4), ((2, 0), -5), ((3, 1), -4), ((4, 1), -8), ((4, 2), -4), ((4, 3), -5), ((3, 3), -1), ((3, 4), -7), ((4, 5), -14), ((5, 5), -13)], Expanded Nodes: 16, Time: 0.0000000000 seconds

Final Results for Algorithm 2:
Total Cost: -13, Path: [((0, 0), 0), ((0, 1), 4), ((1, 1), 3), ((2, 1), 4), ((2, 0), -5), ((3, 1), -4), ((4, 1), -8), ((4, 2), -4), ((4, 3), -5), ((3, 3), -1), ((3, 4), -7), ((4, 5), -14), ((5, 5), -13)], Expanded Nodes: 16, Time: 0.0000000000


### 5.	Comparative Analysis

In [10]:
# Code Block : Print the Time & Space complexity of algorithm 1
print("\nAlgorithm 1 (RBFS) Time & Space Complexity:")
print(f"Time Complexity: {time_1:.10f} seconds")
print(f"Space Complexity (Nodes Expanded): {expanded_nodes_1}")



Algorithm 1 (RBFS) Time & Space Complexity:
Time Complexity: 0.0000000000 seconds
Space Complexity (Nodes Expanded): 29


In [11]:
# Code Block : Print the Time & Space complexity of algorithm 2
print("\nAlgorithm 2 (UCS) Time & Space Complexity:")
print(f"Time Complexity: {time_2:.10f} seconds")
print(f"Space Complexity (Nodes Expanded): {expanded_nodes_2}")



Algorithm 2 (UCS) Time & Space Complexity:
Time Complexity: 0.0000000000 seconds
Space Complexity (Nodes Expanded): 16


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison :
    In the context of the grid-based pathfinding problem, the Recursive Best First Search (RBFS) algorithm efficiently manages memory by using recursion, allowing it to handle smaller grids without significant overhead. 
    However, in more complex city layouts, RBFS may expand a higher number of nodes compared to Uniform Cost Search (UCS), potentially resulting in longer search times due to its heuristic reliance. 
    In contrast, UCS consistently provides the shortest path by evaluating all possible paths based on cumulative costs, but it incurs higher memory usage to store all expanded nodes in its priority queue, particularly in densely populated grids with numerous buildings and roadblocks.

TIME AND SPACE COMPLEXITY FOR ONE OF THE EXECUTIONS IS GIVEN BELOW: THESE TIMIGS ARE GETTING CHANGED FOR THE SUBSEQUENT EXECUTIONS.
 
Algorithm 1 (RBFS) Time & Space Complexity:
Time Complexity: 0.0010004044 seconds
Space Complexity (Nodes Expanded): 29

Algorithm 2 (UCS) Time & Space Complexity:
Time Complexity: 0.0009989738 seconds
Space Complexity (Nodes Expanded): 16
( Space Complexity includes the priority queue also) 

 